In [1]:
import sys
import os
import logging
import timeit
import json
from bson import json_util

import pymongo
from pprint import pprint
import tweepy
import pandas as pd

from MongoConnector import MongoConnector

In [2]:
MONGO_CONFIG  = {  
    'MONGO_COLL': 'tweets',
    'MONGO_DB': 'tweetCorpus',
    'MONGO_HOST': 'localhost',
    'MONGO_PORT': 27017
    }

In [3]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(message)s')
logging.debug('testing...1..2..3..')

2018-09-27 22:48:07,524 testing...1..2..3..


In [4]:
logging.debug("Establishing MongoDB connection with parameters:- {}".format(MONGO_CONFIG))
cursor = MongoConnector(MONGO_CONFIG).__connect__()
logging.info("Established connection...\n")

2018-09-27 22:48:07,530 Establishing MongoDB connection with parameters:- {'MONGO_DB': 'tweetCorpus', 'MONGO_COLL': 'tweets', 'MONGO_PORT': 27017, 'MONGO_HOST': 'localhost'}
2018-09-27 22:48:07,535 Established connection...



In [5]:
# df = pd.DataFrame(columns = ['tweet_id', 'created_at','text', 'o_created_at', 'o_favorite_count', 'o_reply_count', 
#                              'o_retweet_count','lang', 'tweeted_user_id', 'tweeted_user_screen_name', 
#                              'is_retweet', 'retweeted_users', 'retweet_count', 'in_reply_to_user_id', 
#                              'in_reply_to_screen_name', 'reply_count', 'user_mention_ids', 
#                              'user_mention_screen_names', 'hashtag', 'is_quote_status'])

df = pd.DataFrame(columns = ['tweet_id', 'created_at', 'text', 'lang', 'source_id', 'source_name', 'destination_id', 
                             'destination_name', 'link', 'hashtag'])
df
    

,tweet_id,created_at,text,lang,source_id,source_name,destination_id,destination_name,link,hashtag


In [6]:
for tweet in cursor.find({}).limit(1000):
    if tweet.get('retweeted_status') is None:
        print('ss')
        break

ss


In [92]:
tweet_id_list = list()
nodes_id_list = list()
def nodes_edges(cursor):
    counter = 1
    for tweet in cursor.find({}):
#         logging.debug('creating nodes and edges for tweet id: {}'.format(tweet['id_str']))
#         logging.debug('progress: {}%'.format(counter/1000))
        if tweet['id_str'] not in tweet_id_list:
            tweet_id_list.append(tweet['id_str'])
            if tweet.get('retweeted_status') is not None:
#                 df.loc[len(df)] = [tweet['id_str'], tweet['created_at'], tweet['text'], tweet['lang'], 
#                                    tweet['user']['id_str'], tweet['user']['screen_name'], 
#                                    tweet['retweeted_status']['user']['id_str'], 
#                                    tweet['retweeted_status']['user']['screen_name'], 'retweet',
#                                    [i['text'] for i in tweet['retweeted_status']['entities']['hashtags']]]

                if tweet['user']['id_str'] not in nodes_id_list:
                    nodes_id_list.append(tweet['user']['id_str'])
                    nodes_df.loc[len(nodes_df)] = [tweet['user']['id_str'], tweet['user']['screen_name'], 
                                             tweet['user']['created_at'], tweet['user']['friends_count'], 
                                             tweet['user']['followers_count'], tweet['user']['favourites_count'],
                                             tweet['user']['verified']]
                if tweet['retweeted_status']['user']['id_str'] not in nodes_id_list:
                    nodes_id_list.append(tweet['retweeted_status']['user']['id_str'])
                    nodes_df.loc[len(nodes_df)] = [tweet['retweeted_status']['user']['id_str'], 
                                                   tweet['retweeted_status']['user']['screen_name'], 
                                                   tweet['retweeted_status']['user']['created_at'], 
                                                   tweet['retweeted_status']['user']['friends_count'], 
                                                   tweet['retweeted_status']['user']['followers_count'], 
                                                   tweet['retweeted_status']['user']['favourites_count'],
                                                   tweet['retweeted_status']['user']['verified']]
                edges_df.loc[len(edges_df)] = [tweet['user']['id_str'], tweet['retweeted_status']['user']['id_str'],
                                         'directed', 'retweet', 'Null']
#                 try:
#                     mentions =  list(tweet['retweeted_status']['entities']['user_mentions'])
#                 except error as e:
#                     print(e)
#                     counter+=1
#                     continue
#                 else:
#                     for i in mentions:
# #                         df.loc[len(df)] = [tweet['id_str'], tweet['created_at'], tweet['text'], tweet['lang'], 
# #                                            tweet['user']['id_str'], tweet['user']['screen_name'], 
# #                                            i['id_str'], i['screen_name'], 'mention',
# #                                            [i['text'] for i in tweet['retweeted_status']['entities']['hashtags']]]
                        
#                         edges_df.loc[len(edges_df)] = [tweet['user']['id_str'], tweet['retweeted_status']['user']['id_str'],
#                                                  'directed', 'mention', 'Null']
                counter+=1
                    
        

In [101]:
COLL = ['tweets', 'tweets2', 'tweets3', 'tweets4', 'tweets5']

nodes_df = pd.DataFrame(columns=['Id', 'label', 'created_at', 'friends_count', 'followers_count', 'favourites_count',
                                 'verified'])
edges_df = pd.DataFrame(columns=['Source', 'Target', 'Type', 'Link', 'Weight'])
for i in range(0, len(COLL)):
    
    MONGO_CONFIG['MONGO_COLL'] = COLL[i]
    logging.debug("Establishing MongoDB connection with parameters:- {}".format(MONGO_CONFIG))
    cursor = MongoConnector(MONGO_CONFIG).__connect__()
    logging.info("Established connection...\n")
    
    logging.debug('Creating the data frame')
    nodes_edges(cursor)
    logging.debug('Finished creating the data frame')

nodes_df.drop_duplicates(keep=False).to_csv(OUTPUT_DIR+'/nodes_list3.csv', sep=',', encoding='utf-8', index=False)
edges_df.drop_duplicates(keep=False).to_csv(OUTPUT_DIR+'/edges_list3.csv', sep=',', encoding='utf-8', index=False)

nodes_df = nodes_df.iloc[0:0]
edges_df = edges_df.iloc[0:0]

2018-09-28 02:34:02,949 Establishing MongoDB connection with parameters:- {'MONGO_DB': 'tweetCorpus', 'MONGO_COLL': 'tweets', 'MONGO_PORT': 27017, 'MONGO_HOST': 'localhost'}
2018-09-28 02:34:02,952 Established connection...

2018-09-28 02:34:02,953 Creating the data frame
2018-09-28 02:37:57,636 Finished creating the data frame
2018-09-28 02:37:57,637 Establishing MongoDB connection with parameters:- {'MONGO_DB': 'tweetCorpus', 'MONGO_COLL': 'tweets2', 'MONGO_PORT': 27017, 'MONGO_HOST': 'localhost'}
2018-09-28 02:37:57,639 Established connection...

2018-09-28 02:37:57,640 Creating the data frame
2018-09-28 02:40:54,529 Finished creating the data frame
2018-09-28 02:40:54,530 Establishing MongoDB connection with parameters:- {'MONGO_DB': 'tweetCorpus', 'MONGO_COLL': 'tweets3', 'MONGO_PORT': 27017, 'MONGO_HOST': 'localhost'}
2018-09-28 02:40:54,533 Established connection...

2018-09-28 02:40:54,535 Creating the data frame
2018-09-28 02:43:50,604 Finished creating the data frame
2018-09-

In [9]:
len(df)

6051

In [10]:
df.head()

,tweet_id,created_at,text,lang,source_id,source_name,destination_id,destination_name,link,hashtag
0,935833300077285376,Wed Nov 29 11:30:20 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,805159395344715776,xxpxrytuw,1378000488,OriginalFunko,retweet,[StrangerThings]
1,935833300077285376,Wed Nov 29 11:30:20 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,805159395344715776,xxpxrytuw,1378000488,OriginalFunko,mention,[StrangerThings]
2,935833300077285376,Wed Nov 29 11:30:20 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,805159395344715776,xxpxrytuw,89084561,Target,mention,[StrangerThings]
3,935833395241865216,Wed Nov 29 11:30:43 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,600628878,MichaelOrioles,1378000488,OriginalFunko,retweet,[StrangerThings]
4,935833395241865216,Wed Nov 29 11:30:43 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,600628878,MichaelOrioles,1378000488,OriginalFunko,mention,[StrangerThings]


In [11]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

2018-09-27 22:48:43,265 CACHEDIR=/Users/soumithri/.matplotlib
2018-09-27 22:48:43,269 Using fontManager instance from /Users/soumithri/.matplotlib/fontList.json
2018-09-27 22:48:43,427 backend module://ipykernel.pylab.backend_inline version unknown
2018-09-27 22:48:43,441 backend module://ipykernel.pylab.backend_inline version unknown
2018-09-27 22:48:43,448 backend module://ipykernel.pylab.backend_inline version unknown


In [12]:
G = nx.from_pandas_edgelist(df, 'source_name', 'destination_name', edge_attr='link', 
                             create_using=nx.MultiDiGraph())

In [13]:
G.nodes()

NodeView(('luhdorf1984', 'Carol2_18', 'CoupIesPosts', 'Milliestopshate', 'KatharineACox', 'zellecortez_', 'Muronimo', 'IHAB1204x4', 'it0k', 'angelaborden32', 'AliaJHarvey', 'crispy_flake', 'nanycentre', 'TheJakeJeckyll', 'eebeeebee', 'imthephil84', 'EricDurney', 'talkshowstaylor', 'Nicolechi11', 'Make_It_fRAIN', 'RobertsEmma', 'tristinjoachim', 'cecixdreams', 'jsicolts', 'Adriancobbs56', 'polkadot26', 'reb_says_what', 'DrugG4mer', 'causleyconcepts', 'lucky_e_mi', 'G4killer_YT', 'RealBadCopShow', 'Deemy_Cash', 'noah_schnapp', 'fingerguns_amy', 'paolajosephdun6', 'Liz_Scents', 'bitchinbyers', 'WaWayde', 'CadeJonesActor', 'speakerjones', 'IyanCalde', 'phillycouponmom', 'gaea_ca', 'Suarizta_life', 'zoe_music_lover', 'AngelitaMohler', 'dvdromero23', 'RussTaylor622', 'aizorian1', 'danielpsilvdum1', 'GypsyRN212', 'xxxCRGxxx', 'Savannah_Sanai', 'jammosir', 'ChazzSM', 'hisweetlips', 'sg0616', 'ErikD230', 'jarojacobson', 'tonyshybrid', 'QueenLesli', 'mustacheboy583', 'hopperdancingto', 'Andresit

In [14]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family="Arial")
matplotlib.style.use("grayscale")


class dzcnap:
  
  def __init__(self):
    self.attrs = {
        "edge_color" : "gray",
        "edgecolors" : "black", # For Networkx 2.0
        "linewidths" : 1,       # For Networkx 2.0
        "font_family" : "Liberation Sans Narrow",
        "font_size" : 14,
        "node_color" : "pink",
        "node_size" : 700,
        "width" : 2,
    }
    self.thick_attrs = self.attrs.copy()
    self.thick_attrs["alpha"] = 0.5
    self.thick_attrs["width"] = 15

    self.small_attrs = self.attrs.copy()
    self.small_attrs["node_size"] = 50
    self.small_attrs["font_size"] = 10

    self.medium_attrs = self.small_attrs.copy()
    self.medium_attrs["node_size"] = 250

  def set_extent(self, positions, axes, title=None):
      """
      Given node coordinates pos and the subplot,
      calculate and set its extent.
      """
      axes.tick_params(labelbottom="off")
      axes.tick_params(labelleft="off")
      if title:
          axes.set_title(title)

      x_values, y_values = zip(*positions.values())
      x_max = max(x_values)
      y_max = max(y_values)
      x_min = min(x_values)
      y_min = min(y_values)
      x_margin = (x_max - x_min) * 0.1
      y_margin = (y_max - y_min) * 0.1
      try:
          axes.set_xlim(x_min - x_margin, x_max + x_margin)
          axes.set_ylim(y_min - y_margin, y_max + y_margin)
      except AttributeError:
          axes.xlim(x_min - x_margin, x_max + x_margin)
          axes.ylim(y_min - y_margin, y_max + y_margin)

  def plot(self, fname, save = False):
      plt.tight_layout()
      if save:
          plt.savefig("../images/{}.pdf".format(fname), dpi=600)
      plt.show()

In [15]:
import os
os.getcwd()
OUTPUT_DIR = '/Users/soumithri/Desktop/Projects/Thesis/output'
df.to_csv(OUTPUT_DIR+'/tweets.csv', sep='|', encoding='utf-8', index=False)

In [16]:
df1 = df[['source_name', 'destination_name', 'link']].copy()

df1.head()

,source_name,destination_name,link
0,xxpxrytuw,OriginalFunko,retweet
1,xxpxrytuw,OriginalFunko,mention
2,xxpxrytuw,Target,mention
3,MichaelOrioles,OriginalFunko,retweet
4,MichaelOrioles,OriginalFunko,mention


In [17]:
df1.to_csv(OUTPUT_DIR+'/social_connections.csv', sep=',', encoding='utf-8', index=False)

In [20]:
df2 = df[['destination_id', 'destination_name']].rename(columns = {'destination_id' : 'Id',
                                                                    'destination_name': 'Label'})

In [19]:
df3 = df[['source_id', 'source_name']].rename(columns = {'source_id': 'Id', 'source_name': 'Labe'})

,tweet_id,created_at,text,lang,source_id,source_name,destination_id,destination_name,link,hashtag
0,935833300077285376,Wed Nov 29 11:30:20 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,805159395344715776,xxpxrytuw,1378000488,OriginalFunko,retweet,[StrangerThings]
1,935833300077285376,Wed Nov 29 11:30:20 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,805159395344715776,xxpxrytuw,1378000488,OriginalFunko,mention,[StrangerThings]
2,935833300077285376,Wed Nov 29 11:30:20 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,805159395344715776,xxpxrytuw,89084561,Target,mention,[StrangerThings]
3,935833395241865216,Wed Nov 29 11:30:43 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,600628878,MichaelOrioles,1378000488,OriginalFunko,retweet,[StrangerThings]
4,935833395241865216,Wed Nov 29 11:30:43 +0000 2017,RT @OriginalFunko: RT &amp; follow @OriginalFu...,en,600628878,MichaelOrioles,1378000488,OriginalFunko,mention,[StrangerThings]


In [95]:
nodes_df.drop_duplicates(keep=False).to_csv(OUTPUT_DIR+'/nodes_list2.csv', sep=',', encoding='utf-8', index=False)

In [96]:
edges_df.drop_duplicates(keep=False).to_csv(OUTPUT_DIR+'/edges_list2.csv', sep=',', encoding='utf-8', index=False)

In [98]:
len(nodes_df.drop_duplicates(keep=False))

30745

In [88]:
edges_df.head().drop_duplicates()

,Source,Target,Type,Link,Weight
0,805159395344715776,1378000488,directed,retweet,Null
1,600628878,1378000488,directed,retweet,Null
2,167601885,1378000488,directed,retweet,Null
3,30532546,1378000488,directed,retweet,Null
4,2482748221,1378000488,directed,retweet,Null
